In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix
from tqdm import tqdm
import random

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [4]:
"""Dataset paths"""

TRAIN_REAL   = '/kaggle/input/datasets/lightvvcx/full-dataset/FULL_DATASET_FRAMES/train/real'
TRAIN_ATTACK = '/kaggle/input/datasets/lightvvcx/full-dataset/FULL_DATASET_FRAMES/train/attack'
TEST_REAL    = '/kaggle/input/datasets/lightvvcx/full-dataset/FULL_DATASET_FRAMES/test/real'
TEST_ATTACK  = '/kaggle/input/datasets/lightvvcx/full-dataset/FULL_DATASET_FRAMES/test/attack'

In [5]:
"""Dataset class"""

class AntispoofDataset(Dataset):
    def __init__(self, real_path, attack_path, transform=None):
        self.samples = []
        self.transform = transform

        for identity in os.listdir(real_path):
            identity_path = os.path.join(real_path, identity)
            if os.path.isdir(identity_path):
                for img_name in os.listdir(identity_path):
                    if img_name.endswith(('.jpg', '.png', '.jpeg')):
                        self.samples.append((os.path.join(identity_path, img_name), 1))

        for identity in os.listdir(attack_path):
            identity_path = os.path.join(attack_path, identity)
            if os.path.isdir(identity_path):
                for img_name in os.listdir(identity_path):
                    if img_name.endswith(('.jpg', '.png', '.jpeg')):
                        self.samples.append((os.path.join(identity_path, img_name), 0))

        real_count   = sum(1 for _, l in self.samples if l == 1)
        attack_count = sum(1 for _, l in self.samples if l == 0)
        print(f"Loaded {len(self.samples)} samples - Real: {real_count} | Attack: {attack_count}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

In [6]:
"""Data augmentation"""

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [7]:
"""Datasets and loaders"""

train_dataset = AntispoofDataset(TRAIN_REAL, TRAIN_ATTACK, train_transform)
test_dataset  = AntispoofDataset(TEST_REAL,  TEST_ATTACK,  test_transform)

train_loader = DataLoader(train_dataset, batch_size=64,
                          shuffle=True, num_workers=2, pin_memory=True)

test_loader  = DataLoader(test_dataset,  batch_size=64,
                          shuffle=False, num_workers=2, pin_memory=True)

Loaded 2806 samples - Real: 1333 | Attack: 1473
Loaded 1302 samples - Real: 637 | Attack: 665


## BASELINE MODEL without TEXTURE

In [8]:
"""CNN - Baseline (No Texture Branch)"""

class MobileNetBaseline(nn.Module):
    def __init__(self):
        super().__init__()

        backbone = timm.create_model(
            'mobilenetv3_small_100',
            pretrained=True,
            features_only=True
        )

        self.backbone = backbone

        feature_info    = backbone.feature_info
        final_channels  = feature_info[-1]['num_chs']

        self.global_pool = nn.AdaptiveAvgPool2d(1)

        self.classifier = nn.Sequential(
            nn.Linear(final_channels, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)

        final_feat = features[-1]

        main_feat = self.global_pool(final_feat)
        main_feat = main_feat.view(main_feat.size(0), -1)

        out = self.classifier(main_feat)
        return out

In [9]:
"""Model setup"""

model = MobileNetBaseline().to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

NUM_EPOCHS = 20
best_acc = 0

model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Unexpected keys (classifier.bias, classifier.weight, conv_head.bias, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [10]:
"""Training"""

for epoch in range(NUM_EPOCHS):

    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # Validation
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = 100 * correct / total
    auc = roc_auc_score(all_labels, all_preds)

    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print(f"Train Acc: {train_acc:.2f}%")
    print(f"Test Acc:  {test_acc:.2f}% | AUC: {auc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "/kaggle/working/best_model_baseline.pth")
        print("✓ Best model saved")

    scheduler.step()

100%|██████████| 21/21 [00:04<00:00,  4.76it/s]



Epoch 1/20
Train Acc: 85.25%
Test Acc:  71.81% | AUC: 0.8904
✓ Best model saved


100%|██████████| 21/21 [00:02<00:00,  8.77it/s]



Epoch 2/20
Train Acc: 96.19%
Test Acc:  78.49% | AUC: 0.9293
✓ Best model saved


100%|██████████| 21/21 [00:02<00:00,  8.40it/s]


...


Epoch 18/20
Train Acc: 99.89%
Test Acc:  88.86% | AUC: 0.9409


100%|██████████| 21/21 [00:02<00:00,  7.97it/s]



Epoch 19/20
Train Acc: 99.89%
Test Acc:  88.71% | AUC: 0.9418


100%|██████████| 21/21 [00:02<00:00,  8.46it/s]


Epoch 20/20
Train Acc: 99.89%
Test Acc:  88.48% | AUC: 0.9389


## **ENHANCED MODEL with TEXTURE**

In [11]:
"""CNN - Texture Branch Enhanced"""

class MobileNetTexture(nn.Module):
    def __init__(self):
        super().__init__()

        backbone = timm.create_model(
            'mobilenetv3_small_100',
            pretrained=True,
            features_only=True
        )

        self.backbone = backbone

        feature_info = backbone.feature_info
        mid_channels   = feature_info[2]['num_chs']
        final_channels = feature_info[-1]['num_chs']

        self.texture_branch = nn.Sequential(
            nn.Conv2d(mid_channels, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )

        self.global_pool = nn.AdaptiveAvgPool2d(1)

        self.classifier = nn.Sequential(
            nn.Linear(final_channels + 64, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)

        mid_feat   = features[2]
        final_feat = features[-1]

        texture_feat = self.texture_branch(mid_feat)

        main_feat = self.global_pool(final_feat)
        main_feat = main_feat.view(main_feat.size(0), -1)

        combined = torch.cat([main_feat, texture_feat], dim=1)

        out = self.classifier(combined)
        return out


In [13]:
"""Model setup - Texture Branch"""

model = MobileNetTexture().to(device)

criterion  = nn.BCEWithLogitsLoss()
optimizer  = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler  = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

NUM_EPOCHS = 20
best_acc   = 0

Unexpected keys (classifier.bias, classifier.weight, conv_head.bias, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [14]:
"""Training - Texture Branch"""

for epoch in range(NUM_EPOCHS):

    model.train()
    train_loss = 0
    correct    = 0
    total      = 0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds       = (torch.sigmoid(outputs) > 0.5).float()
        correct    += (preds == labels).sum().item()
        total      += labels.size(0)

    train_acc = 100 * correct / total

    model.eval()
    test_loss = 0
    correct   = 0
    total     = 0
    all_preds  = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images).squeeze()
            loss    = criterion(outputs, labels)

            test_loss += loss.item()

            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            correct    += (preds == labels).sum().item()
            total      += labels.size(0)

            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = 100 * correct / total
    auc      = roc_auc_score(all_labels, all_preds)

    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print(f"Train Acc: {train_acc:.2f}%")
    print(f"Test Acc:  {test_acc:.2f}% | AUC: {auc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "/kaggle/working/best_model_texture.pth")
        print("✓ Best model saved")

    scheduler.step()

100%|██████████| 21/21 [00:02<00:00,  8.44it/s]



Epoch 1/20
Train Acc: 84.53%
Test Acc:  83.10% | AUC: 0.9156
✓ Best model saved


100%|██████████| 21/21 [00:02<00:00,  8.16it/s]



Epoch 2/20
Train Acc: 96.01%
Test Acc:  80.03% | AUC: 0.9113


100%|██████████| 21/21 [00:02<00:00,  8.83it/s]


...


Epoch 18/20
Train Acc: 99.89%
Test Acc:  88.63% | AUC: 0.9547


100%|██████████| 21/21 [00:02<00:00,  8.69it/s]



Epoch 19/20
Train Acc: 100.00%
Test Acc:  88.71% | AUC: 0.9566


100%|██████████| 21/21 [00:02<00:00,  7.79it/s]


Epoch 20/20
Train Acc: 99.89%
Test Acc:  88.79% | AUC: 0.9550
